In [8]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import random
import time

In [9]:
def softmax(x):
    z = x - np.max(x)   # overflow protection (softmax(x) = softmax(x - const))
    return np.exp(z) / np.sum(np.exp(z))

activation_functions = {
    'tanh': np.tanh,
    'relu': (lambda x: np.maximum(0, x)),
    'sigmoid': (lambda x: 1 / (1 + np.exp(-x))),
    'linear': (lambda x: x),
    'softmax': softmax
}

**Loading Data**

In [10]:
%%time
t0 = time.time()
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, np.shape(X_train)[1] * np.shape(X_train)[1]).astype(np.float32) / 255.0
X_test = X_test.reshape(-1, np.shape(X_test)[1] * np.shape(X_test)[1]).astype(np.float32) / 255.0
y_train = to_categorical(y_train)   # one-hot encoding
y_test = to_categorical(y_test)  # one-hot encoding

CPU times: total: 359 ms
Wall time: 334 ms


**Network Definition**

In [11]:
class Layer():
    def __init__(self, connectivity_matrix, bias, activation_function, output_layer=False):
        self.connectivity_matrix = connectivity_matrix
        self.bias = bias
        if output_layer:
            assert(activation_function == 'softmax'), 'output layer must use softmax'
        self.activation_function = activation_function
        self.output_layer = output_layer    # final 10 neuron layer (activation fixed to softmax)

        # TODO: tune
        self.mutate_rate_weight = 0.1
        self.mutate_rate_connections = 0.3
        self.mutate_rate_bias = 0.1
        self.mutate_rate_activation_function = 0.1

    def forward(self, input):
        return activation_functions[self.activation_function](self.connectivity_matrix @ input + self.bias)

    def bitstring_mutation(self, param, n_bits=7):
        try:
            # TODO: check if correct (esp. sign)
            bitstring = bin(self.bias)[2:].zfill(n_bits)
            temp = ''
            sign = +1 if random.uniform(0, 1) else -1
            for i in range(len(bitstring)):
                if random.uniform(0, 1) < self.mutate_rate_bias:
                    temp2 = '0' if bitstring[i] == '1' else '1'
                    temp += temp2
                else:
                    temp += bitstring[i]
            return sign * int(temp, 2)
        except (TypeError):
            bitstring = bin(self.bias)[2:].zfill(n_bits)
            temp = ''
            sign = +1 if random.uniform(0, 1) else -1
            for i in range(len(bitstring)):
                if random.uniform(0, 1) < self.mutate_rate_bias:
                    temp2 = '0' if bitstring[i] == '1' else '1'
                    temp += temp2
                else:
                    temp += bitstring[i]
            print(param, temp)


    def mutate(self):
        # TODO: quite arbitrary, inefficient and do negative numbers work?
        # TODO: only works for integer as of now

        # connectivity matrix
        # TODO: row, col correct?
        for row in range(np.shape(self.connectivity_matrix)[0]):
            for col in range(np.shape(self.connectivity_matrix)[1]):
                if random.uniform(0, 1) < self.mutate_rate_connections:
                    self.connectivity_matrix[row][col] = self.bitstring_mutation(self.connectivity_matrix[row][col], n_bits=7)

        # bias
        self.bias = self.bitstring_mutation(self.bias, n_bits=7)

        # activation function
        if not self.output_layer:
            if random.uniform(0, 1) < self.mutate_rate_activation_function:
                self.activation_function = random.choice(list(activation_functions.keys()))

In [12]:
class Network():
    def __init__(self, layers):
        self.layers = layers    # structured (feedforward) array of layers

    def forward(self, input):
        temp = input
        for layer in self.layers:
            temp = layer.forward(temp)
        return temp

    def evaluate(self):  # TODO: slow? (on cpu ~0.55s per call)
        accuracy = 0
        for x, y in zip(X_test, y_test):
            y_pred = np.argmax(self.forward(x.reshape(-1)))  # class with highest value
            y_true = np.argmax(y)
            if y_pred == y_true:
                accuracy += 1
        return accuracy / np.shape(X_test)[0]

    def mutate(self):
        for layer in self.layers:
            layer.mutate()

In [13]:
class Population():
    def __init__(self, size=10, n_survivors=5):
        self.generation = 0
        self.size = size
        self.n_survivors = n_survivors  # TODO: for now must be odd number (1 elite + odd breeders)
        self.elite = None

        # initialization (gaussian) # TODO: better (quite random now - e.g. 32 hidden neurons)
        # TODO: max, min for now 7-bit integers
        self.organisms = []
        for i in range(size):
            self.organisms.append(Network([
                Layer(
                    connectivity_matrix=np.clip(np.random.normal(loc=0.0, scale=1.0, size=(32, 784)), a_min=-128, a_max=128).astype(int),
                    bias=min(128, max(-128, int(np.random.normal(loc=0.0, scale=1.0)))),
                    activation_function='sigmoid',
                    output_layer=False
                ),
                Layer(
                    connectivity_matrix=np.clip(np.random.normal(loc=0.0, scale=1.0, size=(10, 32)), a_min=-128, a_max=128).astype(int),
                    bias=min(128, max(-128, int(np.random.normal(loc=0.0, scale=1.0)))),
                    activation_function='softmax',
                    output_layer=True
                )
            ]))

        self.history = [(max(self.organism_fitness()), self.average_fitness())]   # fitness of population over all generations

    def organism_fitness(self):
        return [organism.evaluate() for organism in self.organisms]

    def average_fitness(self):
        organism_fitness = self.organism_fitness()
        return sum(organism_fitness) / len(organism_fitness)

    def max_fitness(self):
        return max(self.organism_fitness())

    def selection(self):
        organism_fitness = self.organism_fitness()

        # elitism (n=1)
        elite_index = np.argmax(organism_fitness)
        self.elite = self.organisms.pop(elite_index)
        organism_fitness.pop(elite_index)

        probabilities = [fitness / sum(organism_fitness) for fitness in organism_fitness]  # normalized
        survivors = np.random.choice(self.organisms,
                                     size=self.n_survivors - 1,
                                     p=probabilities,
                                     replace=False)    # TODO: works without replacement and p?
        return [survivor for survivor in survivors]

    def crossover(self, parents):
        # TODO: for different type of networks
        # TODO: correct?
        children = []
        while len(children) < (self.size - 1):
            [father, mother] = random.sample(parents + [self.elite], k=2)  # sample without replacement
            layers = []

            # TODO: for now assume same no of layers
            for father_gene, mother_gene in zip(father.layers, mother.layers):
                # full gene crossover # TODO: good?
                child_bias = father_gene.bias if (random.uniform(0, 1) < 0.5) else mother_gene.bias
                child_activation_function = father_gene.activation_function if \
                    (random.uniform(0, 1) < 0.5) else mother_gene.activation_function

                # uniform (bit-wise) crossover # TODO: good?
                child_connectivity_matrix = np.zeros(np.shape(father_gene.connectivity_matrix))
                for row in range(np.shape(child_connectivity_matrix)[0]):
                    for col in range(np.shape(child_connectivity_matrix)[1]):
                        child_connectivity_matrix[row][col] = father_gene.connectivity_matrix[row][col] \
                            if (random.uniform(0, 1) < 0.5) else mother_gene.connectivity_matrix[row][col]

                layers.append(Layer(
                    connectivity_matrix=child_connectivity_matrix,
                    bias=child_bias,
                    activation_function=child_activation_function,
                ))

            children.append(Network(layers))

        return children

    def mutate(self, organisms):
        for organism in organisms:
            organism.mutate()

    def breed(self):
        parents = self.selection()
        children = self.crossover(parents)
        self.mutate(children)
        self.organisms = children + [self.elite]
        self.generation += 1
        self.history.append((self.max_fitness(), self.average_fitness()))

    def plot(self):
        plt.figure()
        plt.plot(np.arange(self.generation + 1), [score[0] for score in self.history],
                 label='max fitness')
        plt.plot(np.arange(self.generation + 1), [score[1] for score in self.history],
                 label='avg fitness', alpha=0.6)
        plt.title('Population fitness' + ' (n=' + str(self.size) + ')')
        plt.xlabel('Generations')
        plt.ylabel('Fitness score (accuracy)')
        plt.legend()
        plt.show()

**Training**

In [14]:
# initialization
GENERATIONS = 50
POPULATION_SIZE = 10
SURVIVORS = 5
population = Population(size=POPULATION_SIZE, n_survivors=SURVIVORS)

In [15]:
# initial population
print('Starting training')
t_training = time.time()
population_fitness = population.organism_fitness()
max_fitness = population.max_fitness()
t2 = time.time()
print('Gen', 0, ':',
      population_fitness, '- max:',
      max_fitness,
      '({}s)'.format(round(t2 - t_training, 2)))

# future populations
for generation in range(1, GENERATIONS):
    # breed new population
    t1 = time.time()
    population.breed()

    # evaluate new population
    ta=time.time()
    population_fitness = population.organism_fitness()
    max_fitness = population.max_fitness()
    print('evaluation time: {}s'.format(round(time.time() - ta, 3)))
    t2 = time.time()

    print('Gen', generation, ':',
          population_fitness, '- max:',
          max_fitness,
          '({}s)'.format(round(t2 - t1, 2)))

print('Finished training ({})'.format(round(time.time() - t_training, 2)))
print('\nTotal computation time: ({}s)'.format(round(time.time() - t0, 2)))

# performance of population
population.plot()

Starting training
Gen 0 : [0.1132, 0.1138, 0.0775, 0.0652, 0.1642, 0.123, 0.131, 0.1208, 0.066, 0.1157] - max: 0.1642 (25.35s)
evaluation time: 21.121s
Gen 1 : [0.1032, 0.0679, 0.0892, 0.0892, 0.0892, 0.0974, 0.101, 0.1028, 0.1028, 0.1642] - max: 0.1642 (54.85s)
evaluation time: 20.286s
Gen 2 : [0.0892, 0.0958, 0.1135, 0.0974, 0.1009, 0.101, 0.1135, 0.0974, 0.1028, 0.1642] - max: 0.1642 (52.0s)
evaluation time: 20.273s
Gen 3 : [0.0974, 0.0892, 0.0974, 0.098, 0.1135, 0.1009, 0.1028, 0.0958, 0.0892, 0.1642] - max: 0.1642 (51.23s)
evaluation time: 20.693s
Gen 4 : [0.0828, 0.1032, 0.1305, 0.098, 0.0892, 0.1135, 0.0716, 0.0958, 0.098, 0.1642] - max: 0.1642 (52.2s)
evaluation time: 20.24s
Gen 5 : [0.1028, 0.0982, 0.0892, 0.104, 0.0864, 0.098, 0.101, 0.1032, 0.0876, 0.1642] - max: 0.1642 (51.47s)
evaluation time: 21.179s
Gen 6 : [0.063, 0.0892, 0.1102, 0.101, 0.1304, 0.0892, 0.098, 0.0624, 0.1028, 0.1642] - max: 0.1642 (53.12s)
evaluation time: 20.324s
Gen 7 : [0.0892, 0.1025, 0.0974, 0.0958,

KeyboardInterrupt: 

**DEBUGGING (multiprocessing)**

In [29]:
from multiprocessing import Process

def test(network):
    network.evaluate()

network1 = Network([
    Layer(
        connectivity_matrix=np.random.normal(loc=0.0, scale=1.0, size=(32, 784)),
        bias=np.random.normal(loc=0.0, scale=1.0),
        activation_function='sigmoid',
        output_layer=False
    ),
    Layer(
        connectivity_matrix=np.random.normal(loc=0.0, scale=1.0, size=(10, 32)),
        bias=np.random.normal(loc=0.0, scale=1.0),
        activation_function='softmax',
        output_layer=True
    )
])
network2 = Network([
    Layer(
        connectivity_matrix=np.random.normal(loc=0.0, scale=1.0, size=(32, 784)),
        bias=np.random.normal(loc=0.0, scale=1.0),
        activation_function='sigmoid',
        output_layer=False
    ),
    Layer(
        connectivity_matrix=np.random.normal(loc=0.0, scale=1.0, size=(10, 32)),
        bias=np.random.normal(loc=0.0, scale=1.0),
        activation_function='softmax',
        output_layer=True
    )
])

Parallel

In [ ]:
%%time
p1 = Process(target=test, args=(network1, ))
p2 = Process(target=test, args=(network2, ))

p1.start()
p2.start()

p1.join()
p2.join()

Synchronous

In [30]:
%%time
test(network1)
test(network2)

CPU times: total: 2.36 s
Wall time: 2.53 s


**DEBUGGING (GPU)**

In [7]:
import cupy as cp

X = cp.asarray(X_test)
Y = cp.asarray(y_test)

def test_gpu(matrix1, bias1, activation1, matrix2, bias2, activation2):
    accuracy = 0
    for x, y in zip(X, Y):
        out = activation_functions[activation2](
                    bias2 + matrix2 @ activation_functions[activation1](
                        bias1 + matrix1 @ x
                    )
                )

        y_pred = cp.argmax(out)  # class with highest value
        y_true = cp.argmax(y)
        if y_pred == y_true:
            accuracy += 1

bias1 = cp.random.normal(loc=0.0, scale=1.0)
matrix1 = cp.random.normal(loc=0.0, scale=1.0, size=(32, 784))
activation1 = 'sigmoid'

bias2 = cp.random.normal(loc=0.0, scale=1.0)
matrix2 = cp.random.normal(loc=0.0, scale=1.0, size=(32, 784))
activation2 = 'softmax'

ModuleNotFoundError: No module named 'cupy'

In [ ]:
X.device

In [ ]:
%%time
test_gpu(matrix1, bias1, activation1, matrix2, bias2, activation2)